#### ref. https://www.tensorflow.org/tutorials/load_data/text?hl=pt-br

In [3]:
!unzip /content/drive/MyDrive/Mentorama/projeto_final/news.zip -d /content/drive/MyDrive/Mentorama/projeto_final/news

Archive:  /content/drive/MyDrive/Mentorama/projeto_final/news.zip
  inflating: /content/drive/MyDrive/Mentorama/projeto_final/news/news.csv  


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf

0 - REAL

1 - FAKE

In [112]:
news_data = pd.read_csv('https://storage.cloudconvert.com/tasks/80fa1fc1-7ad1-4d0c-b31c-9b08051a0923/FakeRecogna_no_removal_words.csv?AWSAccessKeyId=cloudconvert-production&Expires=1663458676&Signature=RLX8O98P7iYvRbGCI6cBsyCWrQM%3D&response-content-disposition=attachment%3B%20filename%3D%22FakeRecogna_no_removal_words.csv%22&response-content-type=text%2Fcsv', index_col=[0])

news_data['label'] = news_data['Classe'].apply(lambda x: 1 if x == 0 else 0)
news_data['text'] = news_data['Noticia'].apply(lambda x: str(x))

news_data = news_data.drop(['Subtitulo', 'Noticia', 'Categoria', 'Data', 'Autor', 'URL', 'Classe'], axis=1)
news_data = news_data.reset_index(drop=True)
news_data.head(1)

,label,text
0,1,bomba lira fraudar o votação brasília explodir...


In [113]:
news_data.dtypes

label     int64
text     object
dtype: object

In [114]:
# categorize labels
news_data['label'] = pd.Categorical(news_data['label'])
news_data['label'] = news_data.label.cat.codes

news_data.head(10)

,label,text
0,1,bomba lira fraudar o votação brasília explodir...
1,0,o whatsapp liberar quinta-feira atualização ap...
2,0,debater propor o cidade o campanha prefeitura ...
3,0,o alesp assembleia legislativo paulo aprovar h...
4,1,“ fauci aprovar o hidroxicloroquina haver ano ...
5,0,dia noite acessório passar o código vestimenta...
6,1,“ pra variar o esquerdo mentir foto e vídeo an...
7,0,o provável origem coronavírus mercar comercial...
8,0,o agência nacional vigilância sanitário anvisa...
9,1,sucessivo falhar umar d atribuir diretamente o...


In [115]:
news_data.shape


(11902, 2)

In [116]:
news_data.dtypes


label      int8
text     object
dtype: object

In [29]:
# drop target
# target = news_data.pop('label')


In [117]:

from sklearn.model_selection import train_test_split

#split train test val
ds_train, ds_test = train_test_split(news_data)

# split validation set
ds_test, ds_val = train_test_split(ds_test)

print(ds_train.shape, ds_test.shape, ds_val.shape)

(8926, 2) (2232, 2) (744, 2)


In [118]:
ds_train

,label,text
9695,1,xuxa ganhar r$ 8.9 milhão hoje o mamata acabar...
11782,0,o amazona liderar o lista desempenhar combater...
3546,1,o jornalista renata vasconcellos parceiro will...
182,1,“ bolsonaro gastar milhão leite condensar corr...
7456,1,o governar austrália recomendar população toma...
...,...,...
4072,0,cientista conicet o agência ciência e técnico ...
8203,0,o comissão temporário ct covid-19 senado feder...
9101,1,blitz o amanhã o cidade parir apreender moto e...
10031,0,o consórcio veículo imprensar divulgar levanta...


In [119]:
ds_train.dtypes


label      int8
text     object
dtype: object

In [120]:
# make a tensor dataset and split into batches

batch_size = 32

raw_train_ds = tf.data.Dataset.from_tensor_slices((ds_train.text.values, ds_train.label.values))
raw_test_ds = tf.data.Dataset.from_tensor_slices((ds_test.text.values, ds_test.label.values))
raw_val_ds = tf.data.Dataset.from_tensor_slices((ds_val.text.values, ds_val.label.values))

raw_train_ds = raw_train_ds.batch(batch_size)
raw_test_ds = raw_test_ds.batch(batch_size)
raw_val_ds = raw_val_ds.batch(batch_size)

In [121]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(10):
    print("Setence: ", text_batch.numpy()[i])
    print("Label:", label_batch.numpy()[i])


Setence:  b'xuxa ganhar r$ 8.9 milh\xc3\xa3o hoje o mamata acabar t\xc3\xa1 explicar o \xc3\xb3dio bolsonaro o xuxa errar pedir o impeachment bolsonaro d o o contar hj ganhar 8,9 milh\xc3\xa3o lei rouanet q ganhar'
Label: 1
Setence:  b'o amazona liderar o lista desempenhar combater o o coronav\xc3\xadrus unidade federa\xc3\xa7\xc3\xa3o mostrar o ranking covid-19 elaborar centrar lideran\xc3\xa7a p\xc3\xbablico clp e obter exclusividade jornal o s. paulo o amazona aparecer topar lista hoje n o in\xc3\xadcio acompanhamento clp 1\xc2\xba abril haver semana notar 71,72 o amazona aparecer distante e colocar o parar e o amap\xc3\xa1 notar 46,66 e 43,23 n ranking o o avalia\xc3\xa7\xc3\xa3o combater doen\xc3\xa7a o amazona 1\xc2\xaa dever evolu\xc3\xa7\xc3\xa3o casar e morte e transpar\xc3\xaancia e isolamento explicar o head competitividade clp jos\xc3\xa9 henrique nascimento o levantamento o amazona taxar mortalidade coronav\xc3\xadrus 7,02 acima m\xc3\xa9dio nacional 4,58 e o propor\xc3\xa

In [122]:
from tensorflow import keras
from keras.layers import TextVectorization

VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 250

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)


In [123]:
# Make a text-only dataset (without labels), then call `TextVectorization.adapt`.
train_text = raw_train_ds.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)


In [124]:
# function to print result
def binary_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return binary_vectorize_layer(text), label

def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text), label


In [125]:
# Retrieve a batch (of 32 reviews and labels) from the dataset.
text_batch, label_batch = next(iter(raw_train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print("Question", first_question)
print("Label", first_label)


Question tf.Tensor(b'xuxa ganhar r$ 8.9 milh\xc3\xa3o hoje o mamata acabar t\xc3\xa1 explicar o \xc3\xb3dio bolsonaro o xuxa errar pedir o impeachment bolsonaro d o o contar hj ganhar 8,9 milh\xc3\xa3o lei rouanet q ganhar', shape=(), dtype=string)
Label tf.Tensor(1, shape=(), dtype=int8)


In [126]:
print("'binary' vectorized question:",
      binary_vectorize_text(first_question, first_label)[0])


'binary' vectorized question: tf.Tensor([[1. 1. 0. ... 0. 0. 0.]], shape=(1, 10000), dtype=float32)


In [127]:
print("'int' vectorized question:",
      int_vectorize_text(first_question, first_label)[0])


'int' vectorized question: tf.Tensor(
[[4933  233   47    1   32   26    2 6890  155  235  199    2 1535   11
     2 4933  512   48    2 1055   11   13    2    2   66 5056  233    1
    32  162 3391 1596  233    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0

In [129]:
print("1289 ---> ", int_vectorize_layer.get_vocabulary()[1289])
print("313 ---> ", int_vectorize_layer.get_vocabulary()[313])
print("Vocabulary size: {}".format(len(int_vectorize_layer.get_vocabulary())))


1289 --->  vagar
313 --->  espalhar
Vocabulary size: 10000


In [152]:
# encoding all datasets

binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_val_ds = raw_val_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

int_train_ds = raw_train_ds.map(int_vectorize_text)
int_val_ds = raw_val_ds.map(int_vectorize_text)
int_test_ds = raw_test_ds.map(int_vectorize_text)


In [153]:
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

binary_train_ds = configure_dataset(binary_train_ds)
binary_val_ds = configure_dataset(binary_val_ds)
binary_test_ds = configure_dataset(binary_test_ds)

int_train_ds = configure_dataset(int_train_ds)
int_val_ds = configure_dataset(int_val_ds)
int_test_ds = configure_dataset(int_test_ds)


In [154]:
from tensorflow.keras import layers
from tensorflow.keras import losses

binary_model = tf.keras.Sequential([layers.Dense(2)])

binary_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

history = binary_model.fit(
    binary_train_ds, validation_data=binary_val_ds, epochs=10)


Epoch 1/10
279/279 [==============================] - 2s 5ms/step - loss: 0.3535 - accuracy: 0.9135 - val_loss: 0.2427 - val_accuracy: 0.9422
Epoch 2/10
279/279 [==============================] - 1s 3ms/step - loss: 0.1671 - accuracy: 0.9681 - val_loss: 0.1809 - val_accuracy: 0.9503
Epoch 3/10
279/279 [==============================] - 2s 6ms/step - loss: 0.1172 - accuracy: 0.9771 - val_loss: 0.1549 - val_accuracy: 0.9516
Epoch 4/10
279/279 [==============================] - 1s 3ms/step - loss: 0.0907 - accuracy: 0.9840 - val_loss: 0.1408 - val_accuracy: 0.9570
Epoch 5/10
279/279 [==============================] - 1s 3ms/step - loss: 0.0736 - accuracy: 0.9870 - val_loss: 0.1322 - val_accuracy: 0.9624
Epoch 6/10
279/279 [==============================] - 1s 3ms/step - loss: 0.0615 - accuracy: 0.9896 - val_loss: 0.1266 - val_accuracy: 0.9610
Epoch 7/10
279/279 [==============================] - 1s 3ms/step - loss: 0.0524 - accuracy: 0.9922 - val_loss: 0.1228 - val_accuracy: 0.9624
Epoch 

In [155]:
print("Linear model on binary vectorized data:")
print(binary_model.summary())


Linear model on binary vectorized data:
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 2)                 20002     
                                                                 
Total params: 20,002
Trainable params: 20,002
Non-trainable params: 0
_________________________________________________________________
None


In [156]:
def create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])
  return model

# `vocab_size` is `VOCAB_SIZE + 1` since `0` is used additionally for padding.
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=2)
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = int_model.fit(int_train_ds, validation_data=int_val_ds, epochs=5)


Epoch 1/5
279/279 [==============================] - 8s 28ms/step - loss: 0.3269 - accuracy: 0.8790 - val_loss: 0.1723 - val_accuracy: 0.9341
Epoch 2/5
279/279 [==============================] - 8s 27ms/step - loss: 0.0777 - accuracy: 0.9721 - val_loss: 0.1690 - val_accuracy: 0.9382
Epoch 3/5
279/279 [==============================] - 8s 28ms/step - loss: 0.0212 - accuracy: 0.9953 - val_loss: 0.2047 - val_accuracy: 0.9288
Epoch 4/5
279/279 [==============================] - 7s 27ms/step - loss: 0.0057 - accuracy: 0.9992 - val_loss: 0.2220 - val_accuracy: 0.9341
Epoch 5/5
279/279 [==============================] - 7s 26ms/step - loss: 0.0021 - accuracy: 0.9998 - val_loss: 0.2394 - val_accuracy: 0.9328


In [157]:
print("ConvNet model on int vectorized data:")
print(int_model.summary())


ConvNet model on int vectorized data:
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 64)          640064    
                                                                 
 conv1d_2 (Conv1D)           (None, None, 64)          20544     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 2)                 130       
                                                                 
Total params: 660,738
Trainable params: 660,738
Non-trainable params: 0
_________________________________________________________________
None


In [158]:
binary_loss, binary_accuracy = binary_model.evaluate(binary_test_ds)
int_loss, int_accuracy = int_model.evaluate(int_test_ds)

print("Binary model accuracy: {:2.2%}".format(binary_accuracy))
print("Int model accuracy: {:2.2%}".format(int_accuracy))


70/70 [==============================] - 1s 7ms/step - loss: 0.1908 - accuracy: 0.9418
Binary model accuracy: 95.97%
Int model accuracy: 94.18%


In [159]:
export_model = tf.keras.Sequential(
    [binary_vectorize_layer, binary_model,
     layers.Activation('softmax')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print("Accuracy: {:2.2%}".format(binary_accuracy))


70/70 [==============================] - 0s 3ms/step - loss: 0.1044 - accuracy: 0.9597
Accuracy: 95.97%


In [160]:
def get_string_labels(predicted_scores_batch):
  predicted_int_labels = tf.argmax(predicted_scores_batch, axis=1)
  # predicted_labels = tf.gather(raw_train_ds, predicted_int_labels)
  # print(predicted_int_labels)
  return predicted_int_labels


In [167]:
inputs = [
    news_data.text[1]
]

predicted_scores = export_model.predict(inputs)
# predicted_labels = get_string_labels(predicted_scores)
print(predicted_scores[0][0])
print(news_data.label[1])


0.81457835
0


In [164]:
export_model.save('/content/drive/MyDrive/Mentorama/projeto_final/modelo_final_0')

#### Save to production

In [149]:
from joblib import dump, load

In [165]:
dump(export_model, '/content/drive/MyDrive/Mentorama/projeto_final/fake-detector_v0.joblib')

['/content/drive/MyDrive/Mentorama/projeto_final/fake-detector_v0.joblib']